In [1]:
# after we install all the packages, import all of them for the use in today's lecture!
# database access
import pandas_datareader as web
import quandl as quandl
import wrds as wrds
# storage and operations
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
import seaborn as sns

# stats
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from scipy import stats
from scipy.stats.mstats import winsorize
import os
from tqdm import tqdm
from zipfile import ZipFile
import seaborn
import matplotlib.pyplot as plt
import linearmodels as lm
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

#portfolio optimization:
# libraries we might use for testing
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
import linearmodels as lm
from linearmodels.panel import compare  
import time
import quantstats as qs

from datetime import datetime

path_factors = Path(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\Master Thesis - Data\Factors')


factor_data = path_factors / 'factor_data.h5'
macro_data = path_factors / 'macro_data.h5'
macro_data_csv = path_factors / 'macro_data.csv'
macro_data_xlsx = path_factors / 'macro_data.xlsx'

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
macro_factors = pd.read_csv(path_factors/'Macro Factors/FRED_MD_Macro_factors 2022-06.csv')

In [4]:
macro_factors.drop(index=0,inplace=True)
macro_factors.drop(index=762,inplace=True)
macro_factors = macro_factors.rename(columns={'sasdate':'date'})
macro_factors.date = pd.to_datetime(macro_factors.date)
macro_factors['date'] = macro_factors['date'].dt.strftime('%Y-%m')
macro_factors = macro_factors.set_index('date')
display(macro_factors)

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,PERMIT,PERMITNE,PERMITMW,PERMITS,PERMITW,ACOGNO,AMDMNOx,ANDENOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,S&P 500,S&P: indust,S&P div yield,S&P PE ratio,FEDFUNDS,CP3Mx,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,COMPAPFFx,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,TWEXAFEGSMTHx,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1959-01,2442.158,2293.2,17.272,2.922664e+05,18235.77392,22.0151,23.3984,22.2848,31.5847,18.6861,38.1504,8.1482,20.1315,12.1386,30.6520,20.8962,19.9176,34.6702,80.1973,1357.0,0.333579,67936.0,63868.0,6.0,16.3,1574.0,1169.0,1396.0,594.0,802.0,291078.0,52478.0,18796.0,713.0,2993.0,14998.0,8740.0,6258.0,33682.0,10774.0,2568.7,5350.3,2418.0,8105.0,39.8,2.5,40.2,1657.0,350.0,452.0,505.0,350.0,NaN,NaN,NaN,NaN,NaN,NaN,14716.48296,NaN,4.262035e+04,8.488956e+04,1.560000,138.9,286.6,987.9,50500.0,18.9,18300.0,35.2130,24.9242,48.96116,0.124965,55.62,59.30,3.158342,18.445741,2.48,3.30,2.82,3.09,3.36,4.01,4.02,4.12,4.87,0.82,0.34,0.61,0.88,1.53,1.54,1.64,2.39,NaN,4.3122,359.8417,2.8065,0.9671,33.100,33.400,30.600,31.600,3.00,32.500,29.010,44.800,29.300,21.100,33.300,38.100,22.900,28.900,30.700,29.600,16.042,56.918,17.791,11.326,2.13,2.45,2.04,NaN,6476.00,12298.00,84.2043,NaN
1959-02,2451.778,2301.5,17.452,2.944247e+05,18369.56308,22.4463,23.7142,22.4778,31.8164,18.7842,38.5150,8.2572,20.7690,12.6977,31.1886,21.2804,19.8634,34.1979,81.4428,1421.0,0.358386,67649.0,63684.0,5.9,15.5,1554.0,1164.0,1277.0,545.0,732.0,282958.0,52688.0,18890.0,704.2,2980.0,15115.0,8839.0,6276.0,33798.0,10816.0,2575.4,5381.3,2420.0,8116.0,39.7,2.6,40.3,1667.0,346.0,469.0,508.0,344.0,NaN,NaN,NaN,NaN,NaN,NaN,15400.24873,NaN,4.367715e+04,8.518181e+04,1.539338,139.4,287.7,992.1,49800.0,18.6,18100.0,35.2201,25.2270,49.51371,0.125765,54.77,58.33,3.219518,18.418119,2.43,3.26,2.70,3.13,3.54,3.96,3.96,4.14,4.89,0.83,0.27,0.70,1.11,1.53,1.53,1.71,2.46,NaN,4.3133,359.8417,2.8093,0.9748,33.200,33.400,30.700,31.400,3.00,32.500,29.000,44.700,29.400,21.200,33.300,38.100,23.000,28.900,30.700,29.600,16.057,56.951,17.798,11.343,2.14,2.46,2.05,NaN,6476.00,12298.00,83.5280,NaN
1959-03,2467.594,2318.5,17.617,2.934187e+05,18523.05762,22.7696,23.8577,22.5882,31.8164,19.1520,38.3327,8.3464,21.3085,13.1980,31.6572,21.6133,19.9989,35.0637,82.4769,1524.0,0.400947,68068.0,64267.0,5.6,15.3,1459.0,1093.0,1210.0,530.0,680.0,260346.0,53014.0,19069.0,704.1,3013.0,15259.0,8965.0,6294.0,33945.0,10873.0,2584.4,5431.6,2430.0,8132.0,40.0,2.8,40.4,1620.0,330.0,413.0,503.0,374.0,NaN,NaN,NaN,NaN,NaN,NaN,15745.42348,NaN,4.478165e+04,8.562019e+04,1.529007,139.7,289.2,998.3,49700.0,18.4,17800.0,35.1304,25.4218,50.00773,0.126123,56.16,59.79,3.151709,18.999351,2.80,3.35,2.80,3.13,3.61,3.99,3.99,4.13,4.85,0.55,0.00,0.33,0.81,1.19,1.19,1.33,2.05,NaN,4.3228,359.8417,2.8127,0.9698,33.200,33.300,30.700,31.500,2.97,32.900,28.970,44.700,29.600,21.300,33.200,38.300,23.000,28.900,30.700,29.600,16.068,57.022,17.785,11.363,2.15,2.45,2.07,NaN,6508.00,12349.00,81.6405,NaN
1959-04,2483.671,2334.9,17.553,2.993228e+05,18534.46600,23.2547,24.2022,22.9191,32.3184,19.2746,38.9708,8.5645,21.8970,13.5953,32.6376,22.0742,2

In [5]:
# some returns need to be divided by 100. The specific factors can be found in the paper in the appendix (p.28) 
macro_factors[['UNRATE','S&P div yield', 'S&P PE ratio','FEDFUNDS','CP3Mx','TB3MS','TB6MS', 'GS1','GS5','GS10','AAA','BAA','COMPAPFFx','TB3SMFFM','TB6SMFFM','T1YFFM','T5YFFM','T10YFFM','AAAFFM','BAAFFM']] = macro_factors[['UNRATE','S&P div yield', 'S&P PE ratio','FEDFUNDS','CP3Mx','TB3MS','TB6MS', 'GS1','GS5','GS10','AAA','BAA','COMPAPFFx','TB3SMFFM','TB6SMFFM','T1YFFM','T5YFFM','T10YFFM','AAAFFM','BAAFFM']]/100
macro_factors

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,PERMIT,PERMITNE,PERMITMW,PERMITS,PERMITW,ACOGNO,AMDMNOx,ANDENOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,S&P 500,S&P: indust,S&P div yield,S&P PE ratio,FEDFUNDS,CP3Mx,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,COMPAPFFx,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,TWEXAFEGSMTHx,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1959-01,2442.158,2293.2,17.272,2.922664e+05,18235.77392,22.0151,23.3984,22.2848,31.5847,18.6861,38.1504,8.1482,20.1315,12.1386,30.6520,20.8962,19.9176,34.6702,80.1973,1357.0,0.333579,67936.0,63868.0,0.060,16.3,1574.0,1169.0,1396.0,594.0,802.0,291078.0,52478.0,18796.0,713.0,2993.0,14998.0,8740.0,6258.0,33682.0,10774.0,2568.7,5350.3,2418.0,8105.0,39.8,2.5,40.2,1657.0,350.0,452.0,505.0,350.0,NaN,NaN,NaN,NaN,NaN,NaN,14716.48296,NaN,4.262035e+04,8.488956e+04,1.560000,138.9,286.6,987.9,50500.0,18.9,18300.0,35.2130,24.9242,48.96116,0.124965,55.62,59.30,0.031583,0.184457,0.0248,0.0330,0.0282,0.0309,0.0336,0.0401,0.0402,0.0412,0.0487,0.0082,0.0034,0.0061,0.0088,0.0153,0.0154,0.0164,0.0239,NaN,4.3122,359.8417,2.8065,0.9671,33.100,33.400,30.600,31.600,3.00,32.500,29.010,44.800,29.300,21.100,33.300,38.100,22.900,28.900,30.700,29.600,16.042,56.918,17.791,11.326,2.13,2.45,2.04,NaN,6476.00,12298.00,84.2043,NaN
1959-02,2451.778,2301.5,17.452,2.944247e+05,18369.56308,22.4463,23.7142,22.4778,31.8164,18.7842,38.5150,8.2572,20.7690,12.6977,31.1886,21.2804,19.8634,34.1979,81.4428,1421.0,0.358386,67649.0,63684.0,0.059,15.5,1554.0,1164.0,1277.0,545.0,732.0,282958.0,52688.0,18890.0,704.2,2980.0,15115.0,8839.0,6276.0,33798.0,10816.0,2575.4,5381.3,2420.0,8116.0,39.7,2.6,40.3,1667.0,346.0,469.0,508.0,344.0,NaN,NaN,NaN,NaN,NaN,NaN,15400.24873,NaN,4.367715e+04,8.518181e+04,1.539338,139.4,287.7,992.1,49800.0,18.6,18100.0,35.2201,25.2270,49.51371,0.125765,54.77,58.33,0.032195,0.184181,0.0243,0.0326,0.0270,0.0313,0.0354,0.0396,0.0396,0.0414,0.0489,0.0083,0.0027,0.0070,0.0111,0.0153,0.0153,0.0171,0.0246,NaN,4.3133,359.8417,2.8093,0.9748,33.200,33.400,30.700,31.400,3.00,32.500,29.000,44.700,29.400,21.200,33.300,38.100,23.000,28.900,30.700,29.600,16.057,56.951,17.798,11.343,2.14,2.46,2.05,NaN,6476.00,12298.00,83.5280,NaN
1959-03,2467.594,2318.5,17.617,2.934187e+05,18523.05762,22.7696,23.8577,22.5882,31.8164,19.1520,38.3327,8.3464,21.3085,13.1980,31.6572,21.6133,19.9989,35.0637,82.4769,1524.0,0.400947,68068.0,64267.0,0.056,15.3,1459.0,1093.0,1210.0,530.0,680.0,260346.0,53014.0,19069.0,704.1,3013.0,15259.0,8965.0,6294.0,33945.0,10873.0,2584.4,5431.6,2430.0,8132.0,40.0,2.8,40.4,1620.0,330.0,413.0,503.0,374.0,NaN,NaN,NaN,NaN,NaN,NaN,15745.42348,NaN,4.478165e+04,8.562019e+04,1.529007,139.7,289.2,998.3,49700.0,18.4,17800.0,35.1304,25.4218,50.00773,0.126123,56.16,59.79,0.031517,0.189994,0.0280,0.0335,0.0280,0.0313,0.0361,0.0399,0.0399,0.0413,0.0485,0.0055,0.0000,0.0033,0.0081,0.0119,0.0119,0.0133,0.0205,NaN,4.3228,359.8417,2.8127,0.9698,33.200,33.300,30.700,31.500,2.97,32.900,28.970,44.700,29.600,21.300,33.200,38.300,23.000,28.900,30.700,29.600,16.068,57.022,17.785,11.363,2.15,2.45,2.07,NaN,6508.00,12349.00,81.6405,NaN
1959-04,2483.671,2334.9,17.553,2.993228

In [6]:
goy_wel_macro_factors = pd.read_excel(path_factors/'Macro Factors/Macro-Predictor data (Goyal and Welch).xlsx')
goy_wel_macro_factors = goy_wel_macro_factors.rename(columns={'yyyymm':'date'})
goy_wel_macro_factors.date = pd.to_datetime(goy_wel_macro_factors.date, format='%Y%m')
goy_wel_macro_factors.date = goy_wel_macro_factors.date.dt.strftime('%Y-%m')
goy_wel_macro_factors = goy_wel_macro_factors.drop(columns=['Index','AAA','BAA', 'csp']) #b/m  'D12','E12',
# goy_wel_macro_factors=goy_wel_macro_factors.set_index('date')
# goy_wel_macro_factors = goy_wel_macro_factors*100
goy_wel_macro_factors

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,date,D12,E12,b/m,tbl,lty,ntis,Rfree,infl,ltr,corpr,svar,CRSP_SPvw,CRSP_SPvwx
0,1871-01,0.260000,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1871-02,0.260000,0.400000,NaN,NaN,NaN,NaN,0.004967,NaN,NaN,NaN,NaN,NaN,NaN
2,1871-03,0.260000,0.400000,NaN,NaN,NaN,NaN,0.004525,NaN,NaN,NaN,NaN,NaN,NaN
3,1871-04,0.260000,0.400000,NaN,NaN,NaN,NaN,0.004252,NaN,NaN,NaN,NaN,NaN,NaN
4,1871-05,0.260000,0.400000,NaN,NaN,NaN,NaN,0.004643,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,2021-08,58.791336,169.833333,0.184756,0.0005,0.0128,0.014846,0.000000,0.002066,-0.0035,-0.0045,0.000602,0.030600,0.029205
1808,2021-09,59.254483,175.370000,0.193036,0.0004,0.0137,0.015598,0.000000,0.002716,-0.0250,-0.0194,0.001393,-0.046076,-0.047152
1809,2021-10,59.635361,182.860000,0.182389,0.0005,0.0158,0.013368,0.000000,0.008308,0.0051,0.0159,0.001151,0.070510,0.069627
1810,2021-11,60.016239,190.350000,0.189455,0.0005,0.0156,0.015640,0.000100,0.004913,0.0210,0.0094,0.001327,-0.007256,-0.008665


In [7]:
macro_factors = pd.merge(goy_wel_macro_factors, macro_factors, how='left', on='date')
macro_factors = macro_factors.loc[(macro_factors.date>=('1977-07')) & (macro_factors.date<=('2020-12'))]


In [8]:
macro_factors

,date,D12,E12,b/m,tbl,lty,ntis,Rfree,infl,ltr,corpr,svar,CRSP_SPvw,CRSP_SPvwx,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,PERMIT,PERMITNE,PERMITMW,PERMITS,PERMITW,ACOGNO,AMDMNOx,ANDENOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,S&P 500,S&P: indust,S&P div yield,S&P PE ratio,FEDFUNDS,CP3Mx,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,COMPAPFFx,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,TWEXAFEGSMTHx,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
1278,1977-07,4.406670,10.516700,0.896783,0.0519,0.0768,0.033182,0.0042,0.004942,-0.0070,-0.0005,0.000595,-0.015453,-0.016589,5189.002,4582.7,36.182,549364.128,62566.65718,48.0819,51.1331,49.4271,66.0636,48.9686,73.0684,23.9664,44.1955,25.9113,81.3620,46.2323,59.1218,66.0849,83.0738,3713.0,0.543711,98913.0,92084.0,0.069,14.3,2859.0,2060.0,1907.0,934.0,973.0,361400.0,82834.0,23106.0,783.5,3981.0,18259.0,11205.0,7054.0,59728.0,16772.0,4032.3,9376.6,4349.0,15299.0,39.9,3.4,40.4,2058.0,217.0,466.0,811.0,564.0,1687.0,165.0,412.0,571.0,539.0,NaN,57418.45517,15694.63390,170707.4345,3.282578e+05,1.518675,320.2,1226.7,2017.6,124100.0,35.3,35000.0,195.1835,162.9974,211.51421,0.127219,100.20,110.10,0.043979,0.097479,0.0542,0.0539,0.0519,0.0540,0.0594,0.0684,0.0733,0.0794,0.0887,-0.0003,-0.0023,-0.0002,0.0052,0.0142,0.0191,0.0252,0.0345,124.943669,2.4105,264.8632,1.7226,1.0612,64.8,64.5,65.1,64.7,13.90,81.5,60.800,79.100,59.000,57.300,64.400,64.500,56.300,59.800,63.300,61.000,32.041,85.536,35.172,24.873,5.97,7.54,5.58,NaN,12900.00,35574.00,230.3024,15.3617
1279,1977-08,4.453330,10.613300,0.926534,0.0549,0.0754,0.033750,0.0044,0.003279,0.0198,0.0136,0.000582,-0.012310,-0.020221,5209.385,4595.6,36.161,550271.675,62512.72666,48.1315,51.3049,49.5337,66.1930,49.0287,73.2358,24.0428,44.1302,25.9834,81.5818,46.4863,58.4133,65.5964,83.2927,3799.0,0.548592,99366.0,92441.0,0.070,13.9,2896.0,2207.0,1836.0,938.0,898.0,367250.0,83075.0,23124.0,768.6,3995.0,18276.0,11220.0,7056.0,59951.0,16834.0,4049.1,9416.7,4368.0,15328.0,39.9,3.5,40.4,2020.0,219.0,437.0,838.0,526.0,1780.0,176.0,446.0,613.0,545.0,NaN,58134.19899,16015.12282,172754.2793,3.310537e+05,1.518675,322.3,1237.0,2024.5,124200.0,35.2,34100.0,197.4881,165.6340,213.86873,0.127538,97.75,107.50,0.045558,0.094381,0.0590,0.0576,0.0549,0.0583,0.0637,0.0703,0.0740,0.0798,0.0882,-0.0014,-0.0041,-0.0007,0.0047,0.0113,0.0150,0.0208,0.0292,125.927215,2.4084,266.6774,1.7397,1.0750,65.2,64.8,65.4,63.9,14.85,81.4,61.100,79.200,58.900,57.700,64.600,64.600,56.600,60.000,63.500,61.200,32.190,85.817,35.270,25.027,5.99,7.56,5.61,89.0,13058.00,36030.00,230.1719,14.4929
1280,1977-09,4.500000,10.710000,0.942263,0.0581,0.0764,0.032368,0.0043,0.003268,-0.0029,-0.0022,0.000463,-0.000825,-0.003084,5241.970,4632.4,36.274,552292.394,62786.52774,48.3308,51.4150,49.6563,66.3105,49.2237,73.2902,24.1349,44.4085,26.0202,81.0545,46.5189,58.0352,65.8161,83.1118,3785.0,0.560658,99453.0,92702.0,0.068,14.0,2883.0,2125.0,1853.0,914.0,939.0,368750.0,83532.0,23244.0,802.4,4023.0,18334.0,11265.0,7069.0,60288.0,16922.0,4065.4,9462.8,4394.0,15403.0,39.8,3.4,40.4,1949.0,173.0,434.0,802.0,540.0,1674.0,171.0,381.0,568.0,554.0,NaN,59601.42679,17326.21387,175404.8767,3.344730e+05,1.529007,324.5,1246.2,2033.0,124400.0,35.2,34500.0,198.9991,167.5557,215.88798,0.127465,96.23,105.90,0.046763

In [9]:
selected_rows = macro_factors[macro_factors.isna().any(axis=1)]
selected_rows

to_drop = []
for c in selected_rows.columns:
    if selected_rows[c].isna().any():
        to_drop.append(c)
        
macro_factors = macro_factors.drop(columns=to_drop)

In [10]:
to_reformat = macro_factors.columns.values.tolist()
to_reformat

['date',
 'D12',
 'E12',
 'b/m',
 'tbl',
 'lty',
 'ntis',
 'Rfree',
 'infl',
 'ltr',
 'corpr',
 'svar',
 'CRSP_SPvw',
 'CRSP_SPvwx',
 'RPI',
 'W875RX1',
 'DPCERA3M086SBEA',
 'CMRMTSPLx',
 'RETAILx',
 'INDPRO',
 'IPFPNSS',
 'IPFINAL',
 'IPCONGD',
 'IPDCONGD',
 'IPNCONGD',
 'IPBUSEQ',
 'IPMAT',
 'IPDMAT',
 'IPNMAT',
 'IPMANSICS',
 'IPB51222S',
 'IPFUELS',
 'CUMFNS',
 'HWI',
 'HWIURATIO',
 'CLF16OV',
 'CE16OV',
 'UNRATE',
 'UEMPMEAN',
 'UEMPLT5',
 'UEMP5TO14',
 'UEMP15OV',
 'UEMP15T26',
 'UEMP27OV',
 'CLAIMSx',
 'PAYEMS',
 'USGOOD',
 'CES1021000001',
 'USCONS',
 'MANEMP',
 'DMANEMP',
 'NDMANEMP',
 'SRVPRD',
 'USTPU',
 'USWTRADE',
 'USTRADE',
 'USFIRE',
 'USGOVT',
 'CES0600000007',
 'AWOTMAN',
 'AWHMAN',
 'HOUST',
 'HOUSTNE',
 'HOUSTMW',
 'HOUSTS',
 'HOUSTW',
 'PERMIT',
 'PERMITNE',
 'PERMITMW',
 'PERMITS',
 'PERMITW',
 'AMDMNOx',
 'ANDENOx',
 'AMDMUOx',
 'BUSINVx',
 'ISRATIOx',
 'M1SL',
 'M2SL',
 'M2REAL',
 'BOGMBASE',
 'TOTRESNS',
 'NONBORRES',
 'BUSLOANS',
 'REALLN',
 'NONREVSL',
 'CONS

In [11]:
macro_factors =macro_factors.rename(columns={'b/m':'bm', 'S&P 500':'sp500','S&P: indust':'sp_indust','S&P div yield':'sp_div_yield','S&P PE ratio':'sp_pe_ratio'}) 
macro_factors.columns.values.tolist()

macro_factors.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

# for c in macro_factors.columns:
#     macro_factors[c] = str(macro_factors[c]).strip()

In [12]:
macro_factors

,date,D12,E12,bm,tbl,lty,ntis,Rfree,infl,ltr,corpr,svar,CRSP_SPvw,CRSP_SPvwx,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,PERMIT,PERMITNE,PERMITMW,PERMITS,PERMITW,AMDMNOx,ANDENOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,sp500,sp_indust,sp_div_yield,sp_pe_ratio,FEDFUNDS,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,TWEXAFEGSMTHx,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
1278,1977-07,4.406670,10.516700,0.896783,0.0519,0.0768,0.033182,0.0042,0.004942,-0.0070,-0.0005,0.000595,-0.015453,-0.016589,5189.002,4582.7,36.182,549364.128,62566.65718,48.0819,51.1331,49.4271,66.0636,48.9686,73.0684,23.9664,44.1955,25.9113,81.3620,46.2323,59.1218,66.0849,83.0738,3713.0,0.543711,98913.0,92084.0,0.069,14.3,2859.0,2060.0,1907.0,934.0,973.0,361400.0,82834.0,23106.0,783.5,3981.0,18259.0,11205.0,7054.0,59728.0,16772.0,4032.3,9376.6,4349.0,15299.0,39.9,3.4,40.4,2058.0,217.0,466.0,811.0,564.0,1687.0,165.0,412.0,571.0,539.0,57418.45517,15694.63390,170707.4345,3.282578e+05,1.518675,320.2,1226.7,2017.6,124100.0,35.3,35000.0,195.1835,162.9974,211.51421,0.127219,100.20,110.10,0.043979,0.097479,0.0542,0.0519,0.0540,0.0594,0.0684,0.0733,0.0794,0.0887,-0.0023,-0.0002,0.0052,0.0142,0.0191,0.0252,0.0345,124.943669,2.4105,264.8632,1.7226,1.0612,64.8,64.5,65.1,64.7,13.90,81.5,60.800,79.100,59.000,57.300,64.400,64.500,56.300,59.800,63.300,61.000,32.041,85.536,35.172,24.873,5.97,7.54,5.58,12900.00,35574.00,230.3024,15.3617
1279,1977-08,4.453330,10.613300,0.926534,0.0549,0.0754,0.033750,0.0044,0.003279,0.0198,0.0136,0.000582,-0.012310,-0.020221,5209.385,4595.6,36.161,550271.675,62512.72666,48.1315,51.3049,49.5337,66.1930,49.0287,73.2358,24.0428,44.1302,25.9834,81.5818,46.4863,58.4133,65.5964,83.2927,3799.0,0.548592,99366.0,92441.0,0.070,13.9,2896.0,2207.0,1836.0,938.0,898.0,367250.0,83075.0,23124.0,768.6,3995.0,18276.0,11220.0,7056.0,59951.0,16834.0,4049.1,9416.7,4368.0,15328.0,39.9,3.5,40.4,2020.0,219.0,437.0,838.0,526.0,1780.0,176.0,446.0,613.0,545.0,58134.19899,16015.12282,172754.2793,3.310537e+05,1.518675,322.3,1237.0,2024.5,124200.0,35.2,34100.0,197.4881,165.6340,213.86873,0.127538,97.75,107.50,0.045558,0.094381,0.0590,0.0549,0.0583,0.0637,0.0703,0.0740,0.0798,0.0882,-0.0041,-0.0007,0.0047,0.0113,0.0150,0.0208,0.0292,125.927215,2.4084,266.6774,1.7397,1.0750,65.2,64.8,65.4,63.9,14.85,81.4,61.100,79.200,58.900,57.700,64.600,64.600,56.600,60.000,63.500,61.200,32.190,85.817,35.270,25.027,5.99,7.56,5.61,13058.00,36030.00,230.1719,14.4929
1280,1977-09,4.500000,10.710000,0.942263,0.0581,0.0764,0.032368,0.0043,0.003268,-0.0029,-0.0022,0.000463,-0.000825,-0.003084,5241.970,4632.4,36.274,552292.394,62786.52774,48.3308,51.4150,49.6563,66.3105,49.2237,73.2902,24.1349,44.4085,26.0202,81.0545,46.5189,58.0352,65.8161,83.1118,3785.0,0.560658,99453.0,92702.0,0.068,14.0,2883.0,2125.0,1853.0,914.0,939.0,368750.0,83532.0,23244.0,802.4,4023.0,18334.0,11265.0,7069.0,60288.0,16922.0,4065.4,9462.8,4394.0,15403.0,39.8,3.4,40.4,1949.0,173.0,434.0,802.0,540.0,1674.0,171.0,381.0,568.0,554.0,59601.42679,17326.21387,175404.8767,3.344730e+05,1.529007,324.5,1246.2,2033.0,124400.0,35.2,34500.0,198.9991,167.5557,215.88798,0.127465,96.23,105.90,0.046763,0.092212,0.0614,0.0581,0.0604,0.0653,0.0704,0.0734,0.0792,0.0880,-0.0033,-0.0010,0.0039,0

In [13]:
################################################# saving  ##################################################
# macro_factors.to_hdf(macro_data, 'macro_factors_monthly', mode='a', data_columns=True, complib='zlib')
############################################################################################################

In [14]:
################################################# saving  ##################################################
macro_factors.to_excel(macro_data_xlsx) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################

In [15]:
################################################# saving  ##################################################
macro_factors.to_csv(macro_data_csv) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################

In [16]:
selected_rows = macro_factors[macro_factors.isna().any(axis=1)]
selected_rows

to_drop = []
for c in selected_rows.columns:
    if selected_rows[c].isna().any():
        print(c)
        


In [17]:
# pd.read_excel(macro_data_xlsx)
pd.read_csv(macro_data_csv)


,Unnamed: 0,date,D12,E12,bm,tbl,lty,ntis,Rfree,infl,ltr,corpr,svar,CRSP_SPvw,CRSP_SPvwx,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,PERMIT,PERMITNE,PERMITMW,PERMITS,PERMITW,AMDMNOx,ANDENOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,sp500,sp_indust,sp_div_yield,sp_pe_ratio,FEDFUNDS,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,TWEXAFEGSMTHx,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
0,1278,1977-07,4.406670,10.516700,0.896783,0.0519,0.0768,0.033182,0.0042,0.004942,-0.0070,-0.0005,0.000595,-0.015453,-0.016589,5189.002,4582.7,36.182,549364.128,62566.65718,48.0819,51.1331,49.4271,66.0636,48.9686,73.0684,23.9664,44.1955,25.9113,81.3620,46.2323,59.1218,66.0849,83.0738,3713.0,0.543711,98913.0,92084.0,0.069,14.3,2859.0,2060.0,1907.0,934.0,973.0,361400.0,82834.0,23106.0,783.5,3981.0,18259.0,11205.0,7054.0,59728.0,16772.0,4032.3,9376.6,4349.0,15299.0,39.9,3.4,40.4,2058.0,217.0,466.0,811.0,564.0,1687.0,165.0,412.0,571.0,539.0,57418.45517,15694.63390,170707.4345,3.282578e+05,1.518675,320.2,1226.7,2017.6,124100.0,35.3,35000.0,195.1835,162.9974,211.51421,0.127219,100.20,110.10,0.043979,0.097479,0.0542,0.0519,0.0540,0.0594,0.0684,0.0733,0.0794,0.0887,-0.0023,-0.0002,0.0052,0.0142,0.0191,0.0252,0.0345,124.943669,2.4105,264.8632,1.7226,1.0612,64.8,64.5,65.1,64.7,13.90,81.5,60.800,79.100,59.000,57.300,64.400,64.500,56.300,59.800,63.300,61.000,32.041,85.536,35.172,24.873,5.97,7.54,5.58,12900.00,35574.00,230.3024,15.3617
1,1279,1977-08,4.453330,10.613300,0.926534,0.0549,0.0754,0.033750,0.0044,0.003279,0.0198,0.0136,0.000582,-0.012310,-0.020221,5209.385,4595.6,36.161,550271.675,62512.72666,48.1315,51.3049,49.5337,66.1930,49.0287,73.2358,24.0428,44.1302,25.9834,81.5818,46.4863,58.4133,65.5964,83.2927,3799.0,0.548592,99366.0,92441.0,0.070,13.9,2896.0,2207.0,1836.0,938.0,898.0,367250.0,83075.0,23124.0,768.6,3995.0,18276.0,11220.0,7056.0,59951.0,16834.0,4049.1,9416.7,4368.0,15328.0,39.9,3.5,40.4,2020.0,219.0,437.0,838.0,526.0,1780.0,176.0,446.0,613.0,545.0,58134.19899,16015.12282,172754.2793,3.310537e+05,1.518675,322.3,1237.0,2024.5,124200.0,35.2,34100.0,197.4881,165.6340,213.86873,0.127538,97.75,107.50,0.045558,0.094381,0.0590,0.0549,0.0583,0.0637,0.0703,0.0740,0.0798,0.0882,-0.0041,-0.0007,0.0047,0.0113,0.0150,0.0208,0.0292,125.927215,2.4084,266.6774,1.7397,1.0750,65.2,64.8,65.4,63.9,14.85,81.4,61.100,79.200,58.900,57.700,64.600,64.600,56.600,60.000,63.500,61.200,32.190,85.817,35.270,25.027,5.99,7.56,5.61,13058.00,36030.00,230.1719,14.4929
2,1280,1977-09,4.500000,10.710000,0.942263,0.0581,0.0764,0.032368,0.0043,0.003268,-0.0029,-0.0022,0.000463,-0.000825,-0.003084,5241.970,4632.4,36.274,552292.394,62786.52774,48.3308,51.4150,49.6563,66.3105,49.2237,73.2902,24.1349,44.4085,26.0202,81.0545,46.5189,58.0352,65.8161,83.1118,3785.0,0.560658,99453.0,92702.0,0.068,14.0,2883.0,2125.0,1853.0,914.0,939.0,368750.0,83532.0,23244.0,802.4,4023.0,18334.0,11265.0,7069.0,60288.0,16922.0,4065.4,9462.8,4394.0,15403.0,39.8,3.4,40.4,1949.0,173.0,434.0,802.0,540.0,1674.0,171.0,381.0,568.0,554.0,59601.42679,17326.21387,175404.8767,3.344730e+05,1.529007,324.5,1246.2,2033.0,124400.0,35.2,34500.0,198.9991,167.5557,215.88798,0.127465,96.23,105.90,0.046763,0.092212,0.0614,0.0581,0.0604,0.0653,0.0704,0.0734,0.0792,0.0880,-0.0033